In [13]:
def load_fruts(path):
    
    from pygame.image import load
    
    try:
        
        banana = load(path+"banana.png")
        cereja = load(path+"cereja.png")
        melancia = load(path+"melancia.png")
        morango = load(path+"morango.png")
        bomba = load(path+"bomba.png")
        
    except Exception as inst:
        print(inst)
        raise inst
    
    return banana,cereja,melancia,morango,bomba
        
def test_load_fruts():
    
    import pytest
    
    with pytest.raises(Exception,match='Unsupported image format'):
        load_fruts("imgs/")
        

In [14]:
def random_number(min_value,max_value,s):
    
    from numpy.random import randint
    
    return randint(low=min_value, high=max_value, size=s)

In [15]:
def random_frut(list_fruts):
    
    
    frut = list_fruts[int(random_number(0,len(list_fruts),1))] 
    ballrect = frut.get_rect(center = (int(random_number(frut.get_width()//2+128,height-frut.get_width()/2,1)),32))
    
    return frut, ballrect

In [16]:
def score_update(ttl=-1,hit=-1,font='rogfonts',size=15,color=(0,0,0)):
    
    from pygame.font import SysFont
    
    txt_ttl = SysFont(font, size).render(str(ttl), True, color)
    txt_hit = SysFont(font, size).render(str(hit), True, color)
    
    return txt_ttl, txt_hit
    

In [55]:
def calLife(life_p,screen,heart):
    
    x=1
    y=20
    for n in range( life_p):
        screen.blit(heart,(x,y))
        x+=27
    

In [64]:
def setscenario(path):
    
    from pygame.image import load
    
    score = load(path+"Score.png")
    heart = load(path+"heart.png")
    
    return score,heart

In [65]:
class player:
    def __init__(self,hit,ttl,life):
        self.hit = hit
        self.ttl = ttl
        self.life = life
    
    def getStatus(self):
        return self.hit,self.ttl,self.life
    
    def setStatus(self,hit,ttl,life):
        self.hit += hit
        self.ttl += ttl
        self.life -= life
        

In [68]:
import sys, pygame
pygame.init()

size = width, height = 760, 480

speed = [0, 1]
white = 225, 225, 225

list_fruts = load_fruts("imgs/")
score,heart = setscenario("imgs_scenario/")

frut, ballrect = random_frut(list_fruts)

if frut: screen = pygame.display.set_mode(size)
else: sys.exit()

ply = player(0,0,4)    

pygame.init() 

while frut:
    
        
        for event in pygame.event.get():
                if event.type == pygame.QUIT: 
                    pygame.quit()
                    sys.exit()

        
    

        ballrect = ballrect.move(speed)
        
        if ballrect.top < 0 or ballrect.bottom > height:
            frut, ballrect = random_frut(list_fruts)
            ply.setStatus(1,0,1)
        
        ttl_p, hit_p, life_p = ply.getStatus()
        
        
        ttl, hit = score_update(ttl_p, hit_p)
        
        screen.fill(white)
        
        screen.blit(score,(-10,-10))
        screen.blit(ttl,(0,65))
        screen.blit(hit,(55,65))
        screen.blit(frut, ballrect)
        calLife(life_p,screen,heart)
        pygame.display.flip()

SystemExit: 